In [4]:
import pandas as pd
kgcandid=pd.read_csv('kgresults.csv')

In [ ]:
import os
import socket
from neo4j import GraphDatabase

# 1. Check if we are in Docker
in_docker = os.path.exists('/.dockerenv')
print(f"🐳 Running inside Docker? {'YES' if in_docker else 'NO'}")

# 2. Hostname Resolution Test
print("\n--- DNS Resolution ---")
for host in ['neo4j', 'graphrag-neo4j', 'localhost']:
    try:
        ip = socket.gethostbyname(host)
        print(f"✅ {host} -> {ip}")
    except Exception as e:
        print(f"❌ {host} -> FAILED ({e})")

# 3. Connection Test
print("\n--- Neo4j Connection ---")
if in_docker:
    # In Docker, we MUST use the service name 'neo4j' or 'graphrag-neo4j'
    target = 'neo4j'
else:
    # Local, we MUST use localhost
    target = 'localhost'

uri = f"bolt://{target}:7687"
print(f"Attempting connection to {uri}...")

try:
    auth = (os.getenv('NEO4J_USER', 'neo4j'), os.getenv('NEO4J_PASSWORD', 'password'))
    driver = GraphDatabase.driver(uri, auth=auth)
    driver.verify_connectivity()
    print("🎉 SUCCESS! Connected.")
    driver.close()
except Exception as e:
    print(f"❌ CONNECTION FAILED: {e}")
    
    print("\n--- TROUBLESHOOTING GUIDE ---")
    if in_docker:
        print("1. If 'neo4j' DNS failed: The containers might not be on the same network named 'graphrag-net'.")
        print("2. If DNS works but Connection Refused: Neo4j container is running but not listening (starting up or crashed).")
    else:
        print("1. You are running LOCALLY. Ensure 'docker-compose up' is running successfully in another terminal.")
        print("2. Check if Neo4j is actually running: 'docker ps' | grep neo4j'")
        print("3. Check Neo4j logs: 'docker logs graphrag-neo4j' (Look for 'Started' message)")
        print("4. If logs say 'Plugin failure', the open-gds.jar might be missing/corrupt in data/neo4j/plugins.")

In [16]:
# [핵심] 실험 모드 vs 타겟 데이터베이스 맵핑
# format: 'csv_mode_name': 'neo4j_database_name'
DB_MAPPING = {
    "BASELINE": "kgnormal",
    "FIBO": "kgfibo"
}

# ==========================================
# 2. 파싱 헬퍼
# ==========================================
def parse_record(record_str):
    try:
        if isinstance(record_str, str):
            return ast.literal_eval(record_str)
        return record_str
    except (ValueError, SyntaxError):
        return {}

# ==========================================
# 3. 특정 DB로 적재하는 함수
# ==========================================
def ingest_to_specific_db(driver, df, mode, target_db):
    print(f"\n🚀 [Mode: {mode}] -> [DB: {target_db}] 적재 시작 (총 {len(df)}건)")
    
    # 쿼리: 노드 생성
    node_query = """
    UNWIND $batch AS row
    MERGE (e:Entity {name: row.text})
    ON CREATE SET e.type = row.type, e.source_mode = $mode
    """
    
    # ⚠️ 핵심 변경 사항: session 생성 시 database 지정
    try:
        with driver.session(database=target_db) as session:
            
            # --- [Step 1] 노드 적재 ---
            batch_nodes = []
            for _, row in df.iterrows():
                parsed_ent = parse_record(row['extracted_entities'])
                entities = parsed_ent.get('extracted_entities', [])
                batch_nodes.extend(entities)
                
                if len(batch_nodes) >= 1000:
                    session.run(node_query, batch=batch_nodes, mode=mode)
                    batch_nodes = []
            
            if batch_nodes:
                session.run(node_query, batch=batch_nodes, mode=mode)
            print(f"   ✅ 노드 생성 완료 (@{target_db})")

            # --- [Step 2] 관계 적재 ---
            rels_by_type = {}
            for _, row in df.iterrows():
                parsed_rel = parse_record(row['linked_relationships'])
                relationships = parsed_rel.get('entity_relationships', [])
                
                for rel in relationships:
                    r_type = rel['relation_type'].strip().upper().replace(" ", "_")
                    if not r_type: continue
                    
                    if r_type not in rels_by_type:
                        rels_by_type[r_type] = []
                    
                    rels_by_type[r_type].append({
                        "source": rel['source_entity'],
                        "target": rel['target_entity']
                    })

            count_rels = 0
            for r_type, batch_data in rels_by_type.items():
                rel_query = f"""
                UNWIND $batch AS row
                MATCH (source:Entity {{name: row.source}})
                MATCH (target:Entity {{name: row.target}})
                MERGE (source)-[:{r_type}]->(target)
                """
                
                batch_size = 1000
                for i in range(0, len(batch_data), batch_size):
                    chunk = batch_data[i:i + batch_size]
                    session.run(rel_query, batch=chunk)
                    count_rels += len(chunk)
                    
            print(f"   🔗 관계 연결 완료 (@{target_db}, {count_rels}건)")
            
    except Exception as e:
        print(f"❌ [Error] '{target_db}' 데이터베이스 접속 실패: {e}")
        print("   (Tip: Enterprise 버전이 아니거나, DB가 생성되지 않았을 수 있습니다.)")

# ==========================================
# 4. 메인 실행 (Multi-Database Routing)
# ==========================================
def run_multidb_ingest(full_df):
    # 드라이버는 한 번만 연결
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    
    # 드라이버 연결 확인
    try:
        driver.verify_connectivity()
    except Exception as e:
        print(f"❌ Neo4j 서버 연결 실패: {e}")
        return

    # 실험 모드별로 데이터 분리 및 적재
    modes = full_df['experiment_mode'].unique()
    print(f"🔎 감지된 실험 모드: {modes}")
    
    for mode in modes:
        if mode not in DB_MAPPING:
            print(f"⚠️ 경고: 모드 '{mode}'에 매핑된 DB가 없습니다. 건너뜁니다.")
            continue
            
        target_db_name = DB_MAPPING[mode]
        subset_df = full_df[full_df['experiment_mode'] == mode]
        
        # 함수 호출 시 드라이버를 재사용
        ingest_to_specific_db(driver, subset_df, mode, target_db_name)

    driver.close()
    print("\n🎉 모든 데이터 적재 완료!")

if 'kgcandid' in locals():
        run_multidb_ingest(kgcandid)
else:
        print("⚠️ 'kgcandid' 데이터프레임이 로드되지 않았습니다.")

In [21]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/Linq-AI-Research/FinDER/data/train-00000-of-00001.parquet")

In [30]:
clean_df = df[
        (df['type'].notnull()) & 
        (df['type'] != 'None') & 
        (df['type'] != '')
    ]

In [39]:
[text for text in clean_df.groupby('type').sample(1).text]

['Impact of non-op adjustments on OI to IBT transition for FY23, ticker in last position.',
 'Tyson Foods global workforce, ticker TSN, shows a contrast between the US vs ex-US.',
 'GPM (Gross Profit/Net Revenues) FY23 ENPH.',
 'Total calc for DaVita pro dev cost per emp.',
 "NVR's headcount decline in 2022 & 2023.",
 'Trend in basic EPS for SBUX over three FYs shows profitability.']